In [10]:
import pandas as pd

word_dic = pd.read_csv('word_dictionary.csv', dtype=str)

# 1. 분리
df_hawkish = pd.DataFrame(word_dic['hawkish'], dtype=str)
df_dovish = pd.DataFrame(word_dic['dovish'], dtype=str)

# 2-1. 내부 중복 제거
df_hawkish = df_hawkish.drop_duplicates()
df_dovish = df_dovish.drop_duplicates()

# 2-2. 공통 단어 추출
common_values = set(df_hawkish['hawkish']).intersection(set(df_dovish['dovish']))
print("공통 단어 개수:", len(common_values))

# 각 컬럼의 전체 단어 개수 확인
print("hawkish 전체 단어 개수:", df_hawkish['hawkish'].nunique())
print("dovish 전체 단어 개수:", df_dovish['dovish'].nunique())

# 교집합에 해당하는 값 삭제
df_hawkish = df_hawkish[~df_hawkish['hawkish'].isin(common_values)]
df_dovish = df_dovish[~df_dovish['dovish'].isin(common_values)]

# 인덱스 재설정 및 결측치 제거
df_hawkish = df_hawkish.reset_index(drop=True)
df_dovish = df_dovish.reset_index(drop=True)
df_hawkish.dropna(inplace=True)
df_dovish.dropna(inplace=True)

print("필터링 후 hawkish 단어 개수:", df_hawkish.shape[0])
print("필터링 후 dovish 단어 개수:", df_dovish.shape[0])

공통 단어 개수: 5086
hawkish 전체 단어 개수: 10000
dovish 전체 단어 개수: 10000
필터링 후 hawkish 단어 개수: 4914
필터링 후 dovish 단어 개수: 4914


In [11]:
df_dovish.head()

,dovish
0,메르스
1,국민투표
2,실효
3,점_고려
4,메르스사태


In [12]:
df_hawkish.head()

,hawkish
0,빅_스텝
1,오미크론
2,인상_기조
3,임금_상승
4,지역_은행


In [13]:
result_df = pd.concat([df_hawkish, df_dovish], axis=1)
result_df.to_csv('word_dictionary_final.csv', index=False, encoding='utf-8-sig')

In [14]:
import pandas as pd

df = pd.read_csv('word_dictionary_final.csv', dtype=str)

def simplify_word(word):

    if pd.isna(word):
        return word
    
    # 밑줄('_') 기준으로 분리
    parts = word.split('_')
    
    # 첫 번째 요소를 시작으로 연속 중복 제거
    new_parts = [parts[0]]
    for part in parts[1:]:
        if part != new_parts[-1]:
            new_parts.append(part)
    
    # 리스트를 띄어쓰기로 연결하여 반환
    return ' '.join(new_parts)

for col in df.columns:
    df[col] = df[col].apply(simplify_word)

# 결과를 새로운 CSV 파일로 저장 (UTF-8 with BOM: 엑셀 등에서 한글 깨짐 방지)
df.to_csv('word_dictionary_cleaned.csv', index=False, encoding='utf-8-sig')